In [1]:
import os
import utils
import configparser

In [2]:
# fix path variable
!docker --version
!python --version

Docker version 20.10.17, build 100c701
Python 3.8.2


### Load config from file

In [3]:
config = configparser.ConfigParser()
config.read('disaster_tweet_detect.config')
image_name = config['default']['image_name']
docker_repo = config['default']['docker_repo']
docker_usr = config['default']['docker_usr']
docker_pwd = config['default']['docker_pwd']
build_req = False
build_image = True
docker_upload = True
run_local = True

### Build requirements.txt
**pipreqs** builds the requirements.txt based on the libraries used in the notebook. **nbconvert** is used to convert the notebook to a script that can be used by **pipreqs**. Additional line added to include **jupyter** as it is needed to run the Bokeh app.

In [4]:
if build_req:
    !pip freeze | grep -v disaster_tweet_detect > requirements.txt

### Build docker image
Image is build with the *image name*. The build will take a while on the first run because it needs to download the base image.

In [5]:
if build_image:
    # build docker image
    !(docker build -q -t {image_name} ..)

sha256:623ad16af2916e24f8585e151086492f4de2673e0e89820f380a07f75184b49c


### Upload to private docker registry

Registry is hosted on a private NAS.

In [6]:
if docker_upload:
    !docker tag {image_name}:latest {docker_repo}/{image_name}:latest
    !docker login {docker_repo} -u {docker_usr} -p {docker_pwd}
    !docker push {docker_repo}/{image_name}:latest

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
The push refers to repository [dockerio.themaplevault.synology.me/disaster_tweet_detect]

b975b5a8: Preparing 
b982a73a: Preparing 
b85609b2: Preparing 
136332b1: Preparing 
4a6544f9: Preparing 
9681971a: Preparing 
4fab7f3e: Preparing 
bf18a086: Preparing 
9b864b54: Preparing 
5a2609e8: Preparing 
3905c937: Preparing 
3cc8b216: Preparing 
4f8b2aac: Preparing 
67b9150b: Preparing 
3914fb92: Preparing 
1f71007e: Preparing 
eb83ea03: Layer already exists latest: digest: sha256:702802ceca2bb4222948a624bd4dccd7b53730e33dd83962a7596e89d66015e9 size: 3886


### Run docker image locally
Docker image will run on a docker server. The Streamlit server will run with its default port (80) but needs to be port forwarded by docker and is defined below.

In [ ]:
if run_local:
    # define exposed port (docker port forwarding)
    port = 8501

    # run docker image
    command = utils.get_docker_run_command(port, image_name,
                                           debug=False)
    # command = utils.get_docker_run_command(port, image_name,
    #                                        local_dir_mount=os.getcwd(),
    #                                        debug=True)

    !{command}

2023-04-09 09:09:18.879 INFO    matplotlib.font_manager: generated new fontManager



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:80

